<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [1]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [2]:
# Constants
spacex_url="https://api.spacexdata.com/v4/launches/past" # SpaceX api

# IBMS static resource about spacex launches; preferred over API, to get testable results
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json' 


# Global variables
BoosterVersion = []

Longitude = []
Latitude = []
LaunchSite = []

PayloadMass = []
Orbit = []

Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []



# From the /rocket column we would like to learn the booster name.
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])


#From the /launchpad we would like to know the name of the launch site being used, the logitude, and the latitude.
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])


# From the /payload we would like to learn the mass of the payload and the orbit that it is going to.
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])


#From "/cores" we would like to learn:
# - The outcome of the landing, 
# - The type of the landing,
# - Number of flights with that core, 
# - Whether gridfins were used, 
# - Wheter the core is reused, 
# - Wheter legs were used, Wheter the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.
def getCoreData(data):
    #Takes the dataset and uses the cores column to call the API and append the data to the lists
    for core in data['cores']:
        if core['core'] != None:
            response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
            Block.append(response['block'])
            ReusedCount.append(response['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])


# 1. Get IBMS static data on SpaceX

In [3]:
# spacexApiResponse = requests.get(spacex_url)
# spacexApiResponse.content

ibmsStaticSpacexResponse = requests.get(static_json_url)
if ibmsStaticSpacexResponse.status_code == 200:
    # Use json_normalize meethod to convert the json result into a dataframe
    dfIbmsSpacex = pd.json_normalize(ibmsStaticSpacexResponse.json())
dfIbmsSpacex[:1]


,static_fire_date_utc,static_fire_date_unix,tbd,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,auto_update,failures,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,False,0.0,5e9d0d95eda69955f709d1eb,False,Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,True,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png,https://images2.imgbox.com/40/e3/GypSkayF_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN


# 2. Processing data to relevant data, transform data in complicated columns

In [4]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
dfIbmsSpacex = dfIbmsSpacex[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
dfIbmsSpacex = dfIbmsSpacex[dfIbmsSpacex['cores'].map(len)==1]
dfIbmsSpacex = dfIbmsSpacex[dfIbmsSpacex['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
dfIbmsSpacex['cores'] = dfIbmsSpacex['cores'].map(lambda x : x[0])
dfIbmsSpacex['payloads'] = dfIbmsSpacex['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
dfIbmsSpacex['date'] = pd.to_datetime(dfIbmsSpacex['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
dfIbmsSpacex = dfIbmsSpacex[dfIbmsSpacex['date'] <= datetime.date(2020, 11, 13)]

dfIbmsSpacex[:1]

,rocket,payloads,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z,2006-03-24


# 3. Fetching data from spacex that is stored as id / foreign keys in IBMs dataset

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.

In [5]:
getBoosterVersion(dfIbmsSpacex)
print( f"Booster Version : {BoosterVersion}" )

"Booster Version : ['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 9', 'Falcon 

In [6]:
getLaunchSite(dfIbmsSpacex)
print( f"Longitude: {Longitude}, \nLatitude: {Latitude}, \nLaunchSite: {LaunchSite}" )


"Longitude: [167.7431292, 167.7431292, 167.7431292, 167.7431292, -80.577366, -80.577366, -80.577366, -120.610829, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -120.610829, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -120.610829, -80.6039558, -80.6039558, -80.6039558, -80.6039558, -80.6039558, -80.6039558, -80.6039558, -120.610829, -80.6039558, -80.6039558, -120.610829, -80.6039558, -120.610829, -80.6039558, -80.6039558, -80.577366, -120.610829, -80.577366, -80.577366, -80.577366, -120.610829, -80.577366, -80.577366, -80.6039558, -80.577366, -80.577366, -80.577366, -120.610829, -80.577366, -80.577366, -120.610829, -80.6039558, -120.610829, -80.577366, -80.577366, -120.610829, -80.6039558, -80.577366, -80.577366, -120.610829, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.577366, -80.6039558, -80.577366, -80.577366, -80.57

In [7]:
getPayloadData(dfIbmsSpacex)
print( f"PayloadMass : {PayloadMass} \nPayloadMass : {PayloadMass} \nOrbit: {Orbit}")

"PayloadMass : [20, None, 165, 200, None, 525, 677, 500, 3170, 3325, 2296, 1316, 4535, 4428, 2216, 2395, 570, 1898, 4707, 2477, 2034, 553, 5271, 3136, 4696, 3100, 2257, 4600, 5500, 9600, 2490, 5600, 5300, None, 6070, 2708, 3669, 9600, 6761, 2910, 475, 4990, 9600, 5200, 3700, 2205, 9600, None, 4230, 6092, 9600, 2760, 350, 3750, 5383.85, 2410, 7076, 9600, 5800, 7060, 2800, 3000, 4000, 2573, 4400, 9600, 12259, 2482, 13200, 1425, 2227.7, 6500, 15600, 5000, 6800, 15600, None, 15600, 15600, 1977, 15600, 15600, 9525, 15600, 15600, 3880, None, 15600, 1600, 15600, 15600, 15600, 15600, 3681] \nPayloadMass : [20, None, 165, 200, None, 525, 677, 500, 3170, 3325, 2296, 1316, 4535, 4428, 2216, 2395, 570, 1898, 4707, 2477, 2034, 553, 5271, 3136, 4696, 3100, 2257, 4600, 5500, 9600, 2490, 5600, 5300, None, 6070, 2708, 3669, 9600, 6761, 2910, 475, 4990, 9600, 5200, 3700, 2205, 9600, None, 4230, 6092, 9600, 2760, 350, 3750, 5383.85, 2410, 7076, 9600, 5800, 7060, 2800, 3000, 4000, 2573, 4400, 9600, 12259,

In [8]:
getCoreData(dfIbmsSpacex)
print( f"Outcome: {Outcome}, \nFlights: {Flights}, \nGridFins: {GridFins}, \nReused : {Reused}, \nLegs: {Legs}, \nLendingPad : {LandingPad}, \nBlock : {Block}, \nReusedCount : {ReusedCount}, \nSerial : {Serial}")

"Outcome: ['None None', 'None None', 'None None', 'None None', 'None None', 'None None', 'None None', 'False Ocean', 'None None', 'None None', 'True Ocean', 'True Ocean', 'None None', 'None None', 'False Ocean', 'False ASDS', 'True Ocean', 'False ASDS', 'None None', 'None ASDS', 'True RTLS', 'False ASDS', 'False ASDS', 'True ASDS', 'True ASDS', 'True ASDS', 'True RTLS', 'True ASDS', 'None ASDS', 'True ASDS', 'True RTLS', 'None None', 'True ASDS', 'True RTLS', 'None None', 'True RTLS', 'True ASDS', 'True ASDS', 'None None', 'True RTLS', 'True ASDS', 'True RTLS', 'True ASDS', 'True ASDS', 'True ASDS', 'True RTLS', 'True Ocean', 'True RTLS', 'True Ocean', 'None None', 'None None', 'None None', 'True ASDS', 'True ASDS', 'None None', 'None None', 'True ASDS', 'True ASDS', 'True ASDS', 'True ASDS', 'True RTLS', 'True ASDS', 'True ASDS', 'False RTLS', 'None None', 'True ASDS', 'True ASDS', 'True ASDS', 'True ASDS', 'True RTLS', 'True RTLS', 'None None', 'True ASDS', 'True ASDS', 'True ASDS', 

# 4. Building a df from collected data

In [37]:
launch_dict = {
    'FlightNumber': list(dfIbmsSpacex['flight_number']),
    'Date': list(dfIbmsSpacex['date']),
    'BoosterVersion':BoosterVersion,
    'PayloadMass':PayloadMass,
    'Orbit':Orbit,
    'LaunchSite':LaunchSite,
    'Outcome':Outcome,
    'Flights':Flights,
    'GridFins':GridFins,
    'Reused':Reused,
    'Legs':Legs,
    'LandingPad':LandingPad,
    'Block':Block,
    'ReusedCount':ReusedCount,
    'Serial':Serial,
    'Longitude': Longitude,
    'Latitude': Latitude
}

launch_dict = pd.DataFrame(launch_dict)
launch_dict.head()


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857


###  Filter the dataframe to only include `Falcon 9` launches


In [38]:
falcon9df = launch_dict.loc[launch_dict["BoosterVersion"] == "Falcon 9"]
falcon9df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


In [39]:
falcon9df.loc[:,'FlightNumber'] = list(range(1, len(falcon9df)+1))
falcon9df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


# 5. Preparing dataframe / Uzdatnianie danych

In [47]:
falcon9df.isnull().sum() 

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [50]:
falcon9df["PayloadMass"].replace(np.nan, falcon9df["PayloadMass"].mean())
falcon9df.isnull().sum() 

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>. Because it wasn't used during landing


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


In [ ]:
falcon9df.to_csv('dataset_part_1.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|


Copyright © 2021 IBM Corporation. All rights reserved.
